In [0]:
import pandas as pd
from datetime import datetime

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS bronze.stock_reference;

In [0]:
nyse_tickers_df = pd.read_csv("/Workspace/Users/vladylenahalchuk@gmail.com/stock-picker/data/nyse_tickers_2025-12-07.csv")

column_map = {
    'Symbol': 'symbol',
    'Name': 'name',
    'Last Sale': 'last_sale',
    'Net Change': 'net_change',
    '% Change': 'pct_change',
    'Market Cap': 'market_cap',
    'Country': 'country',
    'IPO Year': 'ipo_year',
    'Volume': 'volume',
    'Sector': 'sector',
    'Industry': 'industry'
}

nyse_tickers_df = nyse_tickers_df.rename(columns=column_map)

nyse_tickers_df['ingestion_timestamp'] = datetime.now()

spark.createDataFrame(nyse_tickers_df).write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("bronze.stock_reference.company_info")

print(f"Loaded {len(nyse_tickers_df)} companies to bronze.stock_reference.company_info")

In [0]:
%sql
select count(*) as companies_cnt,
       count(distinct country) as countries_cnt,
       count(distinct sector) as sectors_cnt,
       count(distinct industry) as industries_cnt
from bronze.stock_reference.company_info